In [10]:
#Assume that number of nurses and spaces are fixed 
#for D1 and D2

#Jul 24 Morning
    #Removed the doctor decision variable MDi[i] completely.
        #anything i want to do with that decision can be replaced with the binary myDocID[i,d]
#Jul 24 Evening
    #randomly generate collections and doctor availability more like what Yu-Li suggested.
    #changing nursing capacity into a hard constraint as Yu-Li suggested.
#Jul 25 Afternoon
    # The D2 space and nurse constraints have been multiplied by 8.
        # Why?: any of the P[1] nurses available for D2, they can be work 8 D2's in a day.
#Jul 25 Evening
    # Similarly, MD's cannot see more than 8 patients in a day.
    # Reorganized the parameters/accompanying comments so they are more readable and clearer.
#Jul 26 Morning
    # Began implementing chemo. 
        
        #BIG CHANGE: rehauled the entire code's day index j be based on a 7 day week.
            #why?: i'm just too lazy to figure out how to convert a 5 day index a 7 day index halfway into the code
        
            #created a wkdayjbin binary that is 1 if and only if it is a weekday.
            #the D1ij and D2ijd binaries now are bounded above by the weekday identifier binary
                #thus, it will be one if the patient could be scheduled AND it is a weekday.
            #now that j is based on a 7 day week, D1ij now factors in weekends in its computation
                #created D1endi to compute the TRUE end date of stem cell collections
            #the daily sigma computations are now multiplied by the wkdayj binary
                #thus, having no prechemo/stem cell collections on a weekend won't contribute to sum of sigmas
        
        #New parameter: Chem[i] = number of chemo days needed for patient i
            #so, D2[i]+Chem[i] = the last day of chemo
            #constraint: D2[i]+Chem[i] cannot be Thursday or Friday
                #since transfusion can't occur on a weekend
            #as long as ^ is satisfied I won't assume any other scheduling hurdles for now
                #[i.e. space and nurse constraints for chem or infusion]
#Jul 27 Morning
    #Rewrote chem collection to be randomly sampled from the triangular distribution Yu-Li suggested.
    #Cleaned up comments.
        
import gurobipy as gp

import numpy as np

###Parameters

    #number of patients
pat = 20 

    #P[k] = number of nurses available for Dk
P = [10,4] 

    #S[k] number of spaces available for Dk
S = [15,4]

    #number of days for scheduling
tend = 20

    #W[k] = target number of patients in any given days for Dk
W = [7,4] 

    #number of MDs available
docs = 24 

    #what is the day of the week doctor d is available?
weekdayavaild = [np.random.randint(0,5) for d in range(docs)] 

    #generate a list of available days for doctor d
availdj = [[int(j%5 == weekdayavaild[d]) for j in range(tend)]for d in range(docs)] 

    #randomly sample collection amounts for each patient
C = [np.random.choice([1,2,3,4,5], p =[.398,.304,.147,.088,.063]) for i in range(pat)]

    #number of chemo days needed for patient i
        #need distribution from Yu-Li
Chem = [int(np.around(np.random.triangular(1,2,6))) for i in range(pat)]

    # Create model
model = gp.Model()

### Decision variables
D1 = model.addVars(len(C),lb = 0, ub = tend-1, vtype=gp.GRB.INTEGER, name='D1_start')
D2 = model.addVars(len(C),lb = 0, ub = tend-1, vtype=gp.GRB.INTEGER, name='D2')
myDocID = model.addVars(pat,docs,vtype=gp.GRB.BINARY,name="doctor_binary")

### Objective function
    #define varibles   

        # vector of 1 when patient i is assigned to doctor d
    
        # vector of 1 during days stem cell collections occur for patient i on day j.
D1ij = model.addVars(pat,tend,vtype=gp.GRB.BINARY,name="stem_cell_binary")

        # vector of 1 EXACTLY when prechemo occurs for patient i with doctor d on day j.
D2ijd = model.addVars(pat,tend,docs,vtype=gp.GRB.BINARY,name="prechemo_binary")
        
        # totaling how many patients are seen in a day for stem cell collections
T1j = model.addVars(tend,vtype=gp.GRB.INTEGER,name="total_stem_cells_day_j")
T2j = model.addVars(tend,vtype=gp.GRB.INTEGER,name="total_prechemo_day_j")
        
        # target patient error
signsig1j = model.addVars(tend,lb=-2000,vtype=gp.GRB.INTEGER,name = "signed daily error 1")
signsig2j = model.addVars(tend,lb=-2000,vtype=gp.GRB.INTEGER,name = "signed daily error 2")

sig1j = model.addVars(tend,vtype=gp.GRB.INTEGER,name="daily error 1")
sig2j = model.addVars(tend,vtype=gp.GRB.INTEGER,name="daily error 2")

        # total of target patient error
sig1 = model.addVar(vtype=gp.GRB.INTEGER,name = "total error 1")
sig2 = model.addVar(vtype=gp.GRB.INTEGER,name = "total error 2")

        # Q1 gap
            #we need helper variables to make sure a weekend is accounted for
Qi = model.addVars(pat,vtype=gp.GRB.INTEGER,name="Q1")
Qtot = model.addVar(vtype=gp.GRB.INTEGER,name="Q1 sum")

        #Weekday binary creation variables
wkj = model.addVars(tend, vtype=gp.GRB.INTEGER,name = "week")
wkdayj = model.addVars(tend,ub=6, vtype=gp.GRB.INTEGER,name = "weekday")
wkdaybin = model.addVars(tend, vtype=gp.GRB.BINARY,name = "weekend binary")

        #Variables that compute the end date of stem cell collection
N1i = model.addVars(pat,vtype=gp.GRB.INTEGER,name="N1")
R1i = model.addVars(pat,ub = 4,vtype=gp.GRB.INTEGER,name="R1")
N2i = model.addVars(pat,vtype=gp.GRB.INTEGER,name="N2")
R2i = model.addVars(pat,ub = 6,vtype=gp.GRB.INTEGER,name="R2")
D1endi = model.addVars(pat,vtype=gp.GRB.INTEGER,name="D1end")

        # infusion day binary (works just like D1ij,D2ijd)
infbinij = model.addVars(pat,tend,vtype=gp.GRB.BINARY, name ="infusion binary")

### Constraints

    #goal: compute D1endi = last date of stem cell collections
model.addConstrs(D1[i]==7*N1i[i]+R1i[i] for i in range(pat))
model.addConstrs(R1i[i]+C[i] == 7*N2i[i]+R2i[i] for i in range(pat))
model.addConstrs(D1endi[i]==D1[i]+C[i]-1+2*N2i[i] for i in range(pat))

    #goal: compute weekdayj = what day of the week are we on?
model.addConstrs(j == (wkj[j]*7+wkdayj[j]) for j in range(tend))
model.addConstrs((wkdaybin[j]==1)>>(wkdayj[j]<=4) for j in range(tend))
model.addConstrs((wkdaybin[j]==0)>>(wkdayj[j]>=5) for j in range(tend))

    #goal: create D1ij
        #"if D1ij is true, then j is at least the collection date"
model.addConstrs((D1ij[i,j]==1)>>(j >= D1[i]) for i in range(pat) for j in range(tend))
        #"if D1ij is true, then j is at most the end date"
model.addConstrs((D1ij[i,j]==1)>>(j <= D1endi[i]) for i in range(pat) for j in range(tend))
        #"a patient gets as many collections as they actually need"
model.addConstrs(gp.quicksum([D1ij[i,j] for j in range(tend)]) == C[i] for i in range(pat))
        #"a patient cannot get collections on a weekend"
model.addConstrs((D1ij[i,j]<=wkdaybin[j]) for i in range(pat) for j in range(tend))    

    #constraint: every patient must have 1 MD
model.addConstrs(gp.quicksum(myDocID[i,d] for d in range(docs)) ==1  for i in range(pat))

    #goal: create D2ijd
        #"if D2ijd is true, then j is the prechemo date"
model.addConstrs((D2ijd[i,j,d]==1)>>(j == D2[i]) for i in range(pat) for j in range(tend) for d in range(docs))
        # a patient can be scheduled for D2 on a WEEKDAY when THEIR MD is AVAILABLE
model.addConstrs((D2ijd[i,j,d]<=wkdaybin[j]*myDocID[i,d]*availdj[d][j]) for i in range(pat) for j in range(tend) for d in range(docs))
        #"patients MUST have prechemo"
model.addConstrs(gp.quicksum([D2ijd[i,j,d] for j in range(tend) for d in range(docs)]) == 1 for i in range(pat))

    #constraint: infusion happens after a day of rest from chemo and must happen on a weekday
        #subgoal: create infbinij
            #infusion happens after a day of rest from chemo
                #note that D2[i]+Chem[i]+2 is the tentative date of infusion
model.addConstrs((infbinij[i,j]==1)>>(j==(D2[i]+Chem[i]+2)) for i in range(pat) for j in range(tend))
            #infusion must happen on a weekday
model.addConstrs(infbinij[i,j]<=wkdaybin[j] for i in range(pat) for j in range(tend))
            #infusion must happen
model.addConstrs(gp.quicksum(infbinij[i,j] for j in range(tend))==1 for i in range(pat))

    #goal: compute D1 daily counts
model.addConstrs(T1j[j]==gp.quicksum([D1ij[i,j] for i in range(pat)]) for j in range(tend))

    #goal: compute D2 daily counts
model.addConstrs(T2j[j]==gp.quicksum([D2ijd[i,j,d] for i in range(pat) for d in range(docs)]) for j in range(tend))

    #constraint: D2 only happens after D1 finishes
model.addConstrs((D1endi[i])<=(D2[i]-1) for i in range(pat))

    #constraint: we cannot have more patients than spaces for each procedure
model.addConstrs(T1j[j]<=S[0] for j in range(tend))
model.addConstrs(T2j[j]<=8*S[1] for j in range(tend))

    #constraint: we cannot have more patients than nurses for each procedure
model.addConstrs(T1j[j]<=P[0] for j in range(tend))
model.addConstrs((T2j[j]<=8*P[1]) for j in range (tend))

    #constraint: MD can see at most 8 patients in a given day for D2. 
        #[8 hour work day / 1 hour per appt]
model.addConstrs(gp.quicksum(D2ijd[i,j,d]for i in range(pat) for j in range(tend)) <= 8 for d in range(docs))

    #goal: calculate error
        #sig_1
            #create "signsig1j" be the difference of T1j and W0
model.addConstrs(signsig1j[j]==(T1j[j]-W[0])*wkdaybin[j] for j in range(tend))
            #let "sig1j" be the absolute value of signsig1j
model.addConstrs(sig1j[j]==gp.abs_(signsig1j[j]) for j in range(tend))
            #sum up all sig1j's
model.addConstr(sig1 == gp.quicksum([sig1j[j] for j in range(tend)]))

        #sig_2
model.addConstrs(signsig2j[j]==(T2j[j]-W[1])*wkdaybin[j] for j in range(tend))
model.addConstrs(sig2j[j]==gp.abs_(signsig2j[j]) for j in range(tend))
model.addConstr(sig2 == gp.quicksum([sig2j[j] for j in range(tend)]))

    #goal: calculate Q
model.addConstrs(Qi[i]==(D2[i]-D1endi[i]) for i in range(pat))
model.addConstr(Qtot == gp.quicksum(Qi[i] for i in range(pat)))


model.update()

#setObjective
model.setObjective(Qtot+sig1+sig2,gp.GRB.MINIMIZE) 

#optimize the model
model.optimize()

# check the status to see if there is a solution
status = model.getAttr("Status")
if status == 2:
    for j in range(len(C)):
        print("Patient "+str(j)+" schedule (D1,D2):(", D1[j].getAttr("X"), D2[j].getAttr("X"),")")


Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 1147 rows, 11223 columns and 32583 nonzeros
Model fingerprint: 0x6b96ef46
Model has 9640 quadratic constraints
Model has 10880 general constraints
Variable types: 0 continuous, 11223 integer (10900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 7e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+03]
  RHS range        [1e+00, 3e+01]
Presolve added 682 rows and 0 columns
Presolve removed 0 rows and 10195 columns
Presolve time: 2.55s
Presolved: 1829 rows, 1028 columns, 5870 nonzeros
Variable types: 0 continuous, 1028 integer (895 binary)
Found heuristic solution: objective 243.0000000

Root relaxation: objective 1.110000e+02, 583 iterations, 0.13 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     

<gurobi.Var weekend binary[0] (value 1.0)>
<gurobi.Var weekend binary[1] (value 1.0)>
<gurobi.Var weekend binary[2] (value 1.0)>
<gurobi.Var weekend binary[3] (value 1.0)>
<gurobi.Var weekend binary[4] (value 1.0)>
<gurobi.Var weekend binary[5] (value 0.0)>
<gurobi.Var weekend binary[6] (value 0.0)>
<gurobi.Var weekend binary[7] (value 1.0)>
<gurobi.Var weekend binary[8] (value 1.0)>
<gurobi.Var weekend binary[9] (value 1.0)>


In [37]:
for j in range(tend):
    print(D1ij_lb[0,j])

NameError: name 'D1ij_lb' is not defined

In [15]:
for j in range(tend):
    print(sig2j[j])

<gurobi.Var daily error 2[0] (value 4.0)>
<gurobi.Var daily error 2[1] (value 3.0)>
<gurobi.Var daily error 2[2] (value 2.0)>
<gurobi.Var daily error 2[3] (value 1.0)>
<gurobi.Var daily error 2[4] (value -0.0)>
<gurobi.Var daily error 2[5] (value -0.0)>
<gurobi.Var daily error 2[6] (value -0.0)>
<gurobi.Var daily error 2[7] (value 4.0)>
<gurobi.Var daily error 2[8] (value 3.0)>
<gurobi.Var daily error 2[9] (value 3.0)>
<gurobi.Var daily error 2[10] (value 1.0)>
<gurobi.Var daily error 2[11] (value -0.0)>
<gurobi.Var daily error 2[12] (value -0.0)>
<gurobi.Var daily error 2[13] (value -0.0)>
<gurobi.Var daily error 2[14] (value 4.0)>
<gurobi.Var daily error 2[15] (value 3.0)>
<gurobi.Var daily error 2[16] (value 4.0)>
<gurobi.Var daily error 2[17] (value 4.0)>
<gurobi.Var daily error 2[18] (value 4.0)>
<gurobi.Var daily error 2[19] (value -0.0)>


In [17]:
print(D1Tot[0])

NameError: name 'D1Tot' is not defined

In [12]:
for i in range(pat):
    print (sig1)

<gurobi.Var total error 1 (value 51.0)>
<gurobi.Var total error 1 (value 51.0)>
<gurobi.Var total error 1 (value 51.0)>
<gurobi.Var total error 1 (value 51.0)>
<gurobi.Var total error 1 (value 51.0)>
<gurobi.Var total error 1 (value 51.0)>
<gurobi.Var total error 1 (value 51.0)>
<gurobi.Var total error 1 (value 51.0)>
<gurobi.Var total error 1 (value 51.0)>
<gurobi.Var total error 1 (value 51.0)>
<gurobi.Var total error 1 (value 51.0)>
<gurobi.Var total error 1 (value 51.0)>
<gurobi.Var total error 1 (value 51.0)>
<gurobi.Var total error 1 (value 51.0)>
<gurobi.Var total error 1 (value 51.0)>
<gurobi.Var total error 1 (value 51.0)>
<gurobi.Var total error 1 (value 51.0)>
<gurobi.Var total error 1 (value 51.0)>
<gurobi.Var total error 1 (value 51.0)>
<gurobi.Var total error 1 (value 51.0)>


In [13]:
sig2

<gurobi.Var total error 2 (value 40.0)>

In [78]:
Tlist = []
for j in range(tend):
    Tlist.append(T1j[j].getAttr("X"))

In [79]:
Tlist

[5.0, 7.0, 7.0, 6.0, 2.0, 4.0, 2.0, 0.0, 1.0, 1.0, 2.0, 1.0, 1.0, 0.0]

In [18]:
sig1

<gurobi.Var total error 1 (value 16.0)>

In [19]:
T1j[0]

<gurobi.Var total_stem_cells_day_j[0] (value 6.0)>

In [21]:
sum = 0
for j in range(tend):
    sum+=sig1j[j].getAttr("X")
    

In [25]:
sum

213.0

In [22]:
for j in range(tend):
    print(sig1j[j].getAttr("X"))

1.0
1.0
1.0
-0.0
2.0
-0.0
1.0
-0.0
3.0
7.0


<gurobi.Var total error 1 (value 51.0)>

In [33]:
weekdayavaild = [np.random.randint(0,6) for d in range(docs)]


In [37]:
weekdayavaild

[0, 2, 1, 4]

In [68]:
for i in range(pat):
    print ("PATIENT " + str(i))
    for j in range(tend):
        if j%7 == 5 or j%7 == 6:
            print ("WEEKEND "+str(D1ij[i,j].getAttr("X")))
        else:
            print (D1ij[i,j].getAttr("X"))
    print()

PATIENT 0
0.0
-0.0
-0.0
-0.0
-0.0
WEEKEND 0.0
WEEKEND 0.0
-0.0
-0.0
1.0
1.0
1.0
WEEKEND 0.0
WEEKEND 0.0
0.0
0.0
0.0
0.0
0.0
WEEKEND 0.0

PATIENT 1
0.0
0.0
1.0
1.0
-0.0
WEEKEND 0.0
WEEKEND 0.0
-0.0
-0.0
0.0
0.0
0.0
WEEKEND 0.0
WEEKEND 0.0
0.0
0.0
0.0
0.0
0.0
WEEKEND 0.0

PATIENT 2
0.0
-0.0
1.0
1.0
0.0
WEEKEND 0.0
WEEKEND 0.0
-0.0
-0.0
-0.0
0.0
0.0
WEEKEND 0.0
WEEKEND 0.0
0.0
0.0
0.0
0.0
0.0
WEEKEND 0.0

PATIENT 3
0.0
-0.0
-0.0
-0.0
-0.0
WEEKEND 0.0
WEEKEND 0.0
-0.0
-0.0
-0.0
-0.0
-0.0
WEEKEND 0.0
WEEKEND 0.0
1.0
0.0
0.0
0.0
0.0
WEEKEND 0.0

PATIENT 4
0.0
0.0
-0.0
-0.0
1.0
WEEKEND 0.0
WEEKEND 0.0
1.0
1.0
1.0
1.0
0.0
WEEKEND 0.0
WEEKEND 0.0
0.0
0.0
0.0
0.0
0.0
WEEKEND 0.0

PATIENT 5
0.0
0.0
-0.0
-0.0
-0.0
WEEKEND 0.0
WEEKEND 0.0
1.0
0.0
-0.0
-0.0
-0.0
WEEKEND 0.0
WEEKEND 0.0
0.0
0.0
0.0
0.0
0.0
WEEKEND 0.0

PATIENT 6
0.0
0.0
0.0
-0.0
-0.0
WEEKEND 0.0
WEEKEND 0.0
0.0
-0.0
1.0
1.0
0.0
WEEKEND 0.0
WEEKEND 0.0
0.0
0.0
0.0
0.0
0.0
WEEKEND 0.0

PATIENT 7
1.0
-0.0
-0.0
0.0
0.0
WEEKEND 0.0
WEEKEN

In [5]:
Chem

[3.0,
 2.0,
 3.0,
 3.0,
 2.0,
 3.0,
 2.0,
 2.0,
 3.0,
 4.0,
 2.0,
 3.0,
 2.0,
 3.0,
 5.0,
 3.0,
 2.0,
 2.0,
 4.0,
 2.0]

In [6]:
Chem = [np.around(np.random.triangular(1,2,6)) for i in range(pat)]


In [7]:
chem

NameError: name 'chem' is not defined

In [8]:
Chem

[3.0,
 4.0,
 4.0,
 5.0,
 4.0,
 3.0,
 1.0,
 5.0,
 2.0,
 4.0,
 3.0,
 3.0,
 4.0,
 2.0,
 3.0,
 5.0,
 2.0,
 4.0,
 4.0,
 3.0]